# Machine Learning Model

In [ ]:
# %conda install -c conda-forge awswrangler

### Imports

In [ ]:
import pandas as pd
import torch
import boto3
import awswrangler as wr
import warnings

warnings.filterwarnings("ignore")

### Reading Processed data

In [ ]:
source_uris = ['s3://comp333-data/processed_data/combined/run-1702446295691-part-r-00000.gz', 
              's3://comp333-data/processed_data/combined/run-1702446295691-part-r-00001.gz', 
              's3://comp333-data/processed_data/combined/run-1702446295691-part-r-00002.gz']
dfs = wr.s3.read_json(path=source_uris, chunksize=1000, lines=True)
df = pd.concat(dfs)

In [ ]:
print(df.head())

### Creating ML Model

In [ ]:
# Create a model class
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = torch.nn.Linear(4, 1)  # One in and one out

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred